# Most impactful PrimaryDXCodes

We wanted to run an analysis on the PrimaryDX Column to see if there is some kind of correlation between frequent and non frequent flyers

A  primaryDX code essentially measures the main reason for entry of a patient

### initialise libraries

In [1]:
%matplotlib inline
# notebook
import matplotlib.pylab as pylab
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.cluster import KMeans
from collections import Counter
import numpy as np
from sklearn.decomposition import PCA

#make the graphs bigger
pylab.rcParams['figure.figsize'] = (32.0, 24.0)
pylab.rcParams['font.size'] = 24

### Read data from files & setup

In [2]:
cohort = []
ETOH = []
hw = []
labOrders = []
medOrders = []
prescriptions = []
radiology = []
tobacco = []
vitals = []

In [3]:

#converts each row in the file to a list of strings (a list containing a string for each column)
def fileToList(fname):
    dataArray = []
    with open(fname) as f:
        for line in f:
            new = line.strip()
            new = new[1:len(new) - 1]
            new = new.split('","')
            if (len(new) > 1):
                dataArray.append(new)
    return dataArray

In [4]:
#adds each row to a dictionary ordered by de_id 
#where each dict entry is a list of strings(a list containing a string for each column)
def fileToDict(fname):
    dataDict = dict()
    with open(fname) as f:
        for line in f:
            new = line.strip()
            new = new[1:len(new) - 1]
            new = new.split('","')
            if (len(new) > 1):
                if not(new[0] in dataDict):
                    dataDict[new[0]] = [new]
                else:
                    dataDict[new[0]].append(new)
    return dataDict

In [5]:
#get the cohort data containing the PrimaryDXCodes
fname = "2018-4169_Cohort.txt"
cohort = fileToList(fname)

In [9]:
import collections
from datetime import datetime

#returns a dict of lists
    #dict of patients
        #list of admissions
def organiseCohortData(myCohort):
    organised = dict()
    for admission in myCohort:
        if admission[0] in organised:
            organised[admission[0]].append(admission[1:])
        else:
            organised[admission[0]] = [admission[1:]]
            
    
    for p in organised.keys():
        organised[p] = sorted(organised[p], key = lambda x: datetime.strptime(x[8][0:10], '%Y-%m-%d'))
    return organised
    

In [10]:
from datetime import datetime, timedelta

#method for finding frequent flyers where you specify a gap/time frame in days(e.g. 12 months, one month, one week etc)
#and a number of visits, and this method will get frequent flyers for that definition
#uses the organisation of the cohort data specified by the organiseCohortData method (aka dict of lists)
def abstractFreqFlyers(organised, gap, visits):
    frequentFlyers = []
    for p in organised.keys():
        dates = [datetime.strptime(item[8][0:10], '%Y-%m-%d') for item in organised[p]]
        try:
            for i in range(len(dates)):
                modified_date = dates[i] + timedelta(days=gap)
                if dates[i+visits-1] <= modified_date:
                    frequentFlyers.append(p)
                    break
        except:
            pass
    return frequentFlyers

In [11]:
#gets the number of differences between 2 lists
def difference(l1,l2):
    return len([b for a,b in zip(l1,l2) if b != a])  

In [27]:
#given a set of data training points (Xtr) and classes (patientBase), run a number of models  on given X and Y
#use a K-fold algorithm to get error across 4 runs overall
def runModels(Xtr, patientBase, printOn=True):
    kf = KFold(n_splits=5, shuffle=True)
    Ytr = patientBase
    differencesLR = []
    differencesKNN = []
    differencesKM = []

    for train_index, test_index in kf.split(Xtr):
        X_train, X_test = [Xtr[i] for i in train_index], [Xtr[i] for i in test_index]
        y_train, y_test = [Ytr[i] for i in train_index], [Ytr[i] for i in test_index]
    
        if (1 in y_train and 0 in y_train):
            lr = LogisticRegression()
            lr.fit(X_train, y_train)
            #PredLR = lr.predict(X_test)
            differencesLR.append(lr.score(X_test, y_test))
    
            km = KMeans(n_clusters=2)
            km.fit(X_train, y_train)
            PredKM = km.predict(X_test)
            differencesKM.append(1 - difference(PredKM, y_test)/len(y_test))
            
            neigh = KNeighborsClassifier(n_neighbors=5)
            neigh.fit(X_train, y_train)
            differencesKNN.append(neigh.score(X_test, y_test))
    if (printOn): 
        print((differencesLR))
        print((differencesKNN))
        print((differencesKM))
    return (np.mean(differencesLR))

In [20]:
#given an IS9 code, returns an index from 0 to 15, representing the index in the array
#for that given condition represented by the code
def getIndexIS9(code):
    if code >= 0 and code < 140: return 0 #infections
    if code >= 140 and code < 240: return 1 #neoplasms
    if code >= 240 and code < 280: return 3 #immunity/metabolic
    if code >= 280 and code < 290: return 2 #blood 
    if code >= 290 and code < 320: return 4 #mental disorders
    if code >= 320 and code < 390: return 5 #nervous system
    if code >= 390 and code < 460: return 6 #circulation
    if code >= 460 and code < 520: return 7 #respiratry
    if code >= 520 and code < 580: return 8 #digestive system
    if code >= 580 and code < 630: return 15 #genitourinary
    if code >= 630 and code < 680: return 10 #childbirth
    if code >= 680 and code < 710: return 11 #skin
    if code >= 710 and code < 740: return 9 #musculoskeletal
    if code >= 740 and code < 760: return 12 #congenital
    if code >= 760 and code < 780: return 13 #perinatal
    if code >= 800 and code < 1000: return 14 #injury/poisoning

#given an IS10 code, returns an index from 0 to 15, representing the index in the array
#for that given condition represented by the code
def getIndexIS10(code):
    if ((code[0]).lower() == 'a' or (code[0]).lower() == 'b'): return 0
    if ((code[0]).lower() == 'c' or ((code[0]).lower() == 'd' and int(code[1]) < 5)): return 1
    if ((code[0]).lower() == 'd'): return 2
    if ((code[0]).lower() == 'e'): return 3
    if ((code[0]).lower() == 'f'): return 4
    if ((code[0]).lower() == 'g'): return 5
    if ((code[0]).lower() == 'i'): return 6
    if ((code[0]).lower() == 'j'): return 7
    if ((code[0]).lower() == 'k'): return 8
    if ((code[0]).lower() == 'm'): return 9
    if ((code[0]).lower() == 'o'): return 10
    if ((code[0]).lower() == 'l'): return 11
    if ((code[0]).lower() == 'q'): return 12
    if ((code[0]).lower() == 'p'): return 13
    if ((code[0]).lower() == 's' or (code[0]).lower() == 't'): return 14
    if ((code[0]).lower() == 'n'): return 15

#given a list of patient data, we:
# -strip the primaryDXCode
# -convert the DXCode to an index
# -set that index of our row vector to 1
def getFeatures(de_id):
    datalist = organised[de_id]
    codes = [item[10] for item in datalist]
    features = [0] * 16
    for code in codes:
        if (len(code)>0):
            try:
                newcode = float(code)
                features[getIndexIS9(newcode)] = 1
            except:
                if(getIndexIS10(code) is not None): features[getIndexIS10(code)] = 1
    return features

In [39]:
#calculates Xtr and Ytr and runs models 
def runCode(frequents, regs):
    
    #gets a list of vectors representing the PrimaryDXCodes
    freqFeatures = list(map(getFeatures, frequents))
    regFeatures = list(map(getFeatures, regs))

    #X = list of feature vectors, Y = list of 1s and 0s for frequent flyers and non frequent flyers
    Xtr = freqFeatures+regFeatures
    Ytr = [1]*len(freqFeatures) + [0]*len(regFeatures)
    
    #runs models on X and Y
    print("---------------------------------------------------")
    print("Using Labels")
    runModels(Xtr,Ytr)
    print("---------------------------------------------------")
    
    #print most influencial features by running the models on the individual components of the feature vector
    print("5 most influencial features")
    indie = []
    for i in range(len(Xtr[0])):
        indie.append((runModels([[item[i]] for item in Xtr], Ytr, printOn=False), i))
    indie = sorted(indie, key=lambda x: x[0])[::-1]
    indie = indie[0:5]
    print(len(Xtr[0]))
    englishList = ["infections","neoplasms","blood","immunity/metabolic","mental","nervous system","circulatory","respiratory","digestive","muskoskeletal","childbirth","skin","congenital","perinatal","injury/poisoning", "genitourinary"]
    for col in indie:
        print(str(col[0]) + "  " + str(col[1]) + "  " + englishList[col[1]])


In [16]:

#given a list of frequent flyers, and a list of patients, calculate the patients that are not frequent flyers
def retrieveRegulars(org, frequents):
    regularsBase = []
    newOrder = Counter(frequents)
    for k in org.keys():
        if not(k in newOrder):
            regularsBase.append(k)
    
    return list(np.random.choice(regularsBase, len(frequents)))

# Analysis per data column on different definitions of Frequent Flyer

# N visits 2 years after initial visit

Deals with the definiton of frequent flyers that defines a frequent flyer as returning N times within a space of 2 years after a given visit

In [41]:
organised = organiseCohortData(cohort[1:])

In [42]:
freqs = abstractFreqFlyers(organised, 730, 10)
regs = retrieveRegulars(organised, freqs)

runCode(freqs, regs)

---------------------------------------------------
Using Labels
[0.92119089316987746, 0.9159369527145359, 0.93333333333333335, 0.91754385964912277, 0.92105263157894735]
[0.83012259194395799, 0.82661996497373025, 0.84736842105263155, 0.82280701754385965, 0.85263157894736841]
[0.13485113835376528, 0.13485113835376528, 0.12456140350877198, 0.8631578947368421, 0.14035087719298245]
---------------------------------------------------
5 most influencial features
16
0.717395766123  8  digestive
0.70862506529  9  muskoskeletal
0.683734291947  15  genitourinary
0.670761667742  0  infections
0.666538237011  5  nervous


# N visits in 12 months

Deals with the definition that specifies a frequent flyer as visiting N times in the space of 12 months

In [43]:
organised = organiseCohortData(cohort[1:])

In [44]:
# N=3
freqs = abstractFreqFlyers(organised, 365, 3)
regs = retrieveRegulars(organised, freqs)

runCode(freqs, regs)

---------------------------------------------------
Using Labels
[0.84738562091503267, 0.84670697826442232, 0.84572642588658276, 0.84997548619055396, 0.85013891158686061]
[0.51258169934640518, 0.51446314757313283, 0.50596502696519041, 0.50874325870240233, 0.49861088413139404]
[0.46062091503267977, 0.53374734433731, 0.5412649125674129, 0.562183363294656, 0.4482758620689655]
---------------------------------------------------
5 most influencial features
16
0.604621371128  9  muskoskeletal
0.603248560415  8  digestive
0.590665419079  15  genitourinary
0.586089684226  0  infections
0.570270725384  7  respiratory


In [45]:
# N=4
freqs = abstractFreqFlyers(organised, 365, 4)
regs = retrieveRegulars(organised, freqs)

runCode(freqs, regs)


---------------------------------------------------
Using Labels
[0.87978822796636558, 0.87355963874182496, 0.86577390221114914, 0.87418249766427902, 0.86760124610591904]
[0.50264715042042973, 0.51728433509810023, 0.51323575210214889, 0.50700716287760828, 0.51775700934579438]
[0.41139831828090934, 0.5668016194331984, 0.573030208657739, 0.4254126440361258, 0.5710280373831775]
---------------------------------------------------
5 most influencial features
16
0.632241370445  8  digestive
0.631119875118  9  muskoskeletal
0.614115263827  15  genitourinary
0.60925580001  0  infections
0.588513414266  7  respiratory


In [46]:
# N=5
freqs = abstractFreqFlyers(organised, 365, 5)
regs = retrieveRegulars(organised, freqs)

runCode(freqs, regs)


---------------------------------------------------
Using Labels
[0.8932584269662921, 0.88247317322432295, 0.88094021461420546, 0.8799182422074604, 0.89320388349514568]
[0.49387129724208378, 0.510986203372509, 0.51711803781297905, 0.52529381706693923, 0.51251916198262648]
[0.41113381001021454, 0.39039345937659686, 0.38681655595298925, 0.601430761369443, 0.6019417475728155]
---------------------------------------------------
5 most influencial features
16
0.654098876613  8  digestive
0.648169662034  9  muskoskeletal
0.634479616139  15  genitourinary
0.631107107197  0  infections
0.603004588437  14  injury/poisoning


In [47]:
# N=6
freqs = abstractFreqFlyers(organised, 365, 6)
regs = retrieveRegulars(organised, freqs)

runCode(freqs, regs)


---------------------------------------------------
Using Labels
[0.90259230164964654, 0.90259230164964654, 0.90408805031446537, 0.90094339622641506, 0.89229559748427678]
[0.53888452474469761, 0.54595443833464252, 0.52908805031446537, 0.51415094339622647, 0.6470125786163522]
[0.1822466614296936, 0.821681068342498, 0.18003144654088055, 0.8262578616352201, 0.8066037735849056]
---------------------------------------------------
5 most influencial features
16
0.674633164861  8  digestive
0.671175898067  9  muskoskeletal
0.649322157831  15  genitourinary
0.643347685604  0  infections
0.618201569116  5  nervous


# Returned within 28 days ever

Defines a frequent flyer to be anyone who has returned 28 days within their discharge period

In [ ]:
organised = organiseCohortData(cohort[1:])

freqs28days = abstractFreqFlyers(organised, 28, 2)
regs28days = retrieveRegulars(organised, freqs28days)

runCode(freqs28days, regs28days)
